# Social Network Analysis 2

COST Action Training School in Computational Opinion Analysis – COpA

Johannes B. Gruber \| GESIS

# Example: Divided They Blog

## Packages and setup

In [ ]:
import networkx as nx
import igraph as ig
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import seaborn as sns
import requests
import zipfile
import io
import os
import imageio.v2 as imageio
from tqdm import tqdm
import leidenalg
from atproto import Client, models
import pickle
import os
from tqdm import tqdm
import re
from itertools import combinations
import math

## The Blogosphere data

In [ ]:
graph_file = "data/polblogs.zip"
os.makedirs(os.path.dirname(graph_file), exist_ok=True)

if not os.path.exists(graph_file):
    print("Downloading polblogs.zip file...")
    url = "https://public.websites.umich.edu/~mejn/netdata/polblogs.zip"
    response = requests.get(url)
    with open(graph_file, 'wb') as f:
        f.write(response.content)
    print("Download complete.")

# have a quick look at the data description
with zipfile.ZipFile(graph_file, 'r') as zip_ref:
    with zip_ref.open('polblogs.txt') as f:
        print(f.read().decode('utf-8'))

## graph data structure in `R`

Let’s first look at the `igraph` graph class:

In [ ]:
with zipfile.ZipFile(graph_file, 'r') as zip_ref:
    with zip_ref.open('polblogs.gml') as f:
        gml_content = f.read().decode('utf-8')
        with open("data/polblogs.gml", 'w') as f:
            f.write(gml_content)

polblogs_ig = ig.Graph.Read_GML("data/polblogs.gml")

print(type(polblogs_ig))
print(polblogs_ig.summary())

## graph data structure in `R`

We can convert this to a `tidygraph` object:

In [ ]:
# Python equivalent using NetworkX for tidygraph-like functionality
polblogs_nx = ig.Graph.to_networkx(polblogs_ig)
print(type(polblogs_nx))
print(f"NetworkX Graph with {polblogs_nx.number_of_nodes()} nodes and {polblogs_nx.number_of_edges()} edges")
print(f"Directed: {polblogs_nx.is_directed()}")

## graph data structure in `R`/`Python`

Both `igraph` and `tbl_graph` objects essentially consist of two linked
tables containing <span class="kn-pink">nodes</span> (aka vertices) and
<span class="kn-pink">edges</span>.

In [ ]:
nodes_df = pd.DataFrame({
    'id': range(polblogs_ig.vcount()),
    'value': polblogs_ig.vs['value'],
    'name': polblogs_ig.vs['label']
})
print("\nNodes:")
print(nodes_df.head())

In [ ]:
edges_df = pd.DataFrame({
    'from': [e.source for e in polblogs_ig.es],
    'to': [e.target for e in polblogs_ig.es]
})
print("\nEdges:")
print(edges_df.head())

## working with graph data structures in `R`

We can look up the variables (like political class) for any given node
by filtering. For example, let’s see the node with ID 30:

In [ ]:
for vertex in polblogs_ig.vs:
    if vertex['id'] == 30:
        print(vertex)

## working with graph data structures in `R`

If we want to see how many left and right blogs are in the data, we can
use `count`. But only after converting the nodes to a `data.frame`!

In [ ]:
left_blogs = []
right_blogs = []
for vertex in polblogs_ig.vs:
    if vertex['value'] == 0:
        left_blogs.append(vertex)
    else:
        right_blogs.append(vertex)

print(f"{len(left_blogs)} left blogs")
print(f"{len(right_blogs)} right blogs")

## working with graph data structures in `R`

The `value` variable is named terribly and stored in a strange format.
Let’s change that using some more tidyverse functions that work
out-of-the-box with `tidygraph` graphs:

In [ ]:
ideology = []
for vertex in polblogs_ig.vs:
    if vertex['value'] == 0:
        ideology.append("left")
    else:
        ideology.append("right")

polblogs_ig.vs['ideology'] = ideology

print(polblogs_ig.vs[1])

It’s not necessary to store ideology as a factor, but generally good
practice in `R`. Whenever you have a variable with just a few repeating
character values, a factor is more efficient.

## first insights

We can answer some initial questions about the data:

-   how many left and right blogs are there?

In [ ]:
polblogs_ig.vs['ideology'].count('left')
polblogs_ig.vs['ideology'].count('right')

## first insights

We can answer some initial questions about the data:

-   how many connections (edges) do nodes have to other nodes?

In [ ]:
from_counts = edges_df['from'].value_counts().head(10)
print("Top nodes by outgoing connections:")
print(from_counts)

Let’s have a closer look at the top node:

In [ ]:
print(nodes_df[nodes_df['id'] == 854])

In [ ]:
to_counts = edges_df['to'].value_counts().head(10)
print("\nTop nodes by incoming connections:")
print(to_counts)

Let’s have a closer look at the top node:

In [ ]:
print(nodes_df[nodes_df['id'] == 154])

## first insights

We can answer some initial questions about the data:

-   do left and right blogs reference each other?

In [ ]:
nodes_df = pd.DataFrame({
    'id': range(polblogs_ig.vcount()),
    'value': polblogs_ig.vs['value'],
    'name': polblogs_ig.vs['label'],
    'ideology': polblogs_ig.vs['ideology']
}) 

# Map ideology to from and to nodes
edges_df['from_ideo'] = edges_df['from'].map(nodes_df['ideology'])
edges_df['to_ideo'] = edges_df['to'].map(nodes_df['ideology'])

# Count combinations of from_ideo and to_ideo
print(edges_df.groupby(['from_ideo', 'to_ideo']).size().reset_index(name='n'))

## visualising graphs in `R`

-   `ggraph` inherits the idea of a grammar of graphics from `ggplot`
-   hence, we build up plots in layers with visual aesthetics mapped to
    data
-   the difference is, we address map data from the nodes and edges
    table

<figure>
<img src="https://ggraph.data-imaginist.com/reference/figures/logo.png"
alt="ggraph" />
<figcaption aria-hidden="true">ggraph</figcaption>
</figure>

## visualising graphs in `Python`

<figure>
<img src="https://networkx.org/_static/networkx_logo.svg"
alt="NetworkX" />
<figcaption aria-hidden="true">NetworkX</figcaption>
</figure>

-   Basic network visualization using NetworkX and matplotlib

In [ ]:
plt.figure(figsize=(12, 8))

# Get node colors based on ideology
node_colors = ['#2F357E' if polblogs_ig.vs[i]['ideology'] == 'left' else '#D72F32'
               for i in range(len(polblogs_ig.vs))]

nx.draw_kamada_kawai(polblogs_nx, with_labels=False, node_color=node_colors)
plt.show()

## Recreate the plot from Adamic and Glance (2005)

In [ ]:
plt.figure(figsize=(15, 12))

# Calculate in-degree centrality (referenced)
in_degrees = polblogs_nx.in_degree()
referenced = {node: degree for node, degree in in_degrees}

# Remove isolated nodes
non_isolated_nodes = [node for node in polblogs_nx.nodes() if polblogs_nx.degree(node) > 0]
subgraph = polblogs_nx.subgraph(non_isolated_nodes)

# Prepare edge colors based on ideology connections
edge_colors = []
for edge in subgraph.edges():
    from_node, to_node = edge
    from_ideo = polblogs_ig.vs[from_node]['ideology']
    to_ideo = polblogs_ig.vs[to_node]['ideology']
    
    if from_ideo == "left" and to_ideo == "left":
        edge_colors.append("#2F357E")
    elif from_ideo == "right" and to_ideo == "right":
        edge_colors.append("#D72F32")
    else:  # bipartisan connections
        edge_colors.append("#f4c23b")

# Prepare node properties
node_colors = ['#2F357E' if polblogs_ig.vs[node]['ideology'] == 'left' else '#D72F32' 
               for node in subgraph.nodes()]
node_sizes = [referenced.get(node, 1) * 10 + 20 for node in subgraph.nodes()]

# Draw the network
nx.draw_kamada_kawai(subgraph, with_labels=False, node_color=node_colors,
                     edge_color=edge_colors)

plt.show()

## volatile layouts

One thing that always makes me cautious about interpreting network plots
is how volatile the placement of nodes in the plot is and how much it
can trick you into finding a pattern where none exists. So let’s look at
an experiment:

Prepare data for plotting:

In [ ]:
np.random.seed(1)

# Sample 25% of nodes randomly for faster plotting
sample_size = int(0.25 * polblogs_nx.number_of_nodes())
sampled_nodes = np.random.choice(list(polblogs_nx.nodes()), size=sample_size, replace=False)
plot_subgraph = polblogs_nx.subgraph(sampled_nodes)

# Remove isolated nodes
non_isolated = [node for node in plot_subgraph.nodes() if plot_subgraph.degree(node) > 0]
plot_subgraph = plot_subgraph.subgraph(non_isolated)

# Calculate referenced (in-degree)
in_degrees_sample = plot_subgraph.in_degree()

Get all available layouts:

In [ ]:
nx_layouts = {
    'circular': nx.circular_layout,
    'random': nx.random_layout,
    'shell': nx.shell_layout,
    'spring': nx.spring_layout,
    'kamada_kawai': nx.kamada_kawai_layout,
    'planar': nx.planar_layout,
    'spectral': nx.spectral_layout
}

## volatile layouts

In [ ]:
# Create plots with different layouts to show volatility
os.makedirs("media/layouts_py/", exist_ok=True)

def create_network_plot(subgraph, layout_func, layout_name, save_path):
    plt.figure(figsize=(10, 8))
    
    try:
        if layout_name == 'planar':
            # Check if graph is planar
            if nx.is_planar(subgraph):
                pos = layout_func(subgraph)
            else:
                pos = nx.spring_layout(subgraph)  # fallback
        else:
            pos = layout_func(subgraph)
    except:
        pos = nx.spring_layout(subgraph)  # fallback for any errors
    
    # Prepare edge colors
    edge_colors = []
    for edge in subgraph.edges():
        from_node, to_node = edge
        if from_node < len(polblogs_ig.vs) and to_node < len(polblogs_ig.vs):
            from_ideo = polblogs_ig.vs[from_node]['ideology']
            to_ideo = polblogs_ig.vs[to_node]['ideology']
            
            if from_ideo == "left" and to_ideo == "left":
                edge_colors.append("#2F357E")
            elif from_ideo == "right" and to_ideo == "right":
                edge_colors.append("#D72F32")
            else:
                edge_colors.append("#f4c23b")
        else:
            edge_colors.append("#gray")
    
    # Prepare node properties
    node_colors = []
    node_sizes = []
    for node in subgraph.nodes():
        if node < len(polblogs_ig.vs):
            ideology = polblogs_ig.vs[node]['ideology']
            node_colors.append('#2F357E' if ideology == 'left' else '#D72F32')
            node_sizes.append(in_degrees_sample.get(node, 1) * 20 + 30)
        else:
            node_colors.append('gray')
            node_sizes.append(30)
    
    # Draw network
    nx.draw_kamada_kawai(subgraph, with_labels=False, node_color=node_colors,
                         edge_color=edge_colors)
    # nx.draw_networkx_edges(subgraph, pos, edge_color=edge_colors, alpha=0.6, 
    #                       width=0.5, arrows=True, arrowsize=8)
    # nx.draw_networkx_nodes(subgraph, pos, node_color=node_colors, 
    #                       node_size=node_sizes, alpha=0.8, edgecolors='black')
    
    plt.title(f"Layout: {layout_name}", fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()

# Generate plots for different layouts
layout_files = []
for layout_name, layout_func in nx_layouts.items():
    plot_path = f"media/layouts_py/network_{layout_name}.png"
    print(plot_path)
    create_network_plot(plot_subgraph, layout_func, layout_name, plot_path)
    layout_files.append(plot_path)
    print(f"Created plot with {layout_name} layout")

# Create GIF from the images
if layout_files:
    images = [imageio.imread(f) for f in layout_files if os.path.exists(f)]
    if images:
        imageio.mimsave("media/layouts_python.gif", images, duration=1.5)
        print("Created animated GIF showing layout volatility")

## community detection

In the previous figure we used the political orientation of blogs
manually assigned by Adamic and Glance (2005). Usually, we want to
detect communities from the network structure itself. We learned about
the <span class="kn-pink">Louvain</span> and
<span class="kn-pink">Leiden</span> algorithms (and about their
downsides). So let’s start with these.

In [ ]:
# Community detection using Louvain and Leiden algorithms
import leidenalg
import networkx.algorithms.community as nx_comm

# Convert to undirected graph for community detection
polblogs_undirected = polblogs_nx.to_undirected()

# Louvain community detection using NetworkX
louvain_communities = nx_comm.louvain_communities(polblogs_undirected, seed=42)

# Create group assignments for Louvain
group_louvain = {}
for i, community in enumerate(louvain_communities):
    for node in community:
        group_louvain[node] = i

# Leiden community detection using igraph and leidenalg
leiden_partition = leidenalg.find_partition(polblogs_ig, leidenalg.ModularityVertexPartition)

# Create group assignments for Leiden
group_leiden = {}
for i, community in enumerate(leiden_partition):
    for node in community:
        group_leiden[node] = i

# Add community assignments to nodes dataframe
nodes_df['group_louvain'] = nodes_df['id'].map(group_louvain)
nodes_df['group_leiden'] = nodes_df['id'].map(group_leiden)

print("Nodes with community assignments:")
print(nodes_df.head(10))

print(f"\nNumber of Louvain communities: {len(louvain_communities)}")
print(f"Number of Leiden communities: {len(set(leiden_partition.membership))}")

## community detection

In [ ]:
# Plot network with Louvain communities
plt.figure(figsize=(15, 12))

# Create layout
non_isolated_nodes = [node for node in polblogs_undirected.nodes() if polblogs_undirected.degree(node) > 0]
subgraph_undirected = polblogs_undirected.subgraph(non_isolated_nodes[1:100])
# pos = nx.draw_kamada_kawai(subgraph_undirected)

# Prepare node colors based on Louvain communities
louvain_colors = plt.cm.Set3(np.linspace(0, 1, len(louvain_communities)))
node_colors_louvain = []
for node in subgraph_undirected.nodes():
    community_id = group_louvain.get(node, 0)
    node_colors_louvain.append(louvain_colors[community_id % len(louvain_colors)])

# Prepare node sizes based on in-degree
in_degrees_full = polblogs_nx.in_degree()

# Draw the network
nx.draw_kamada_kawai(subgraph_undirected, edge_color='black',
                     width=0.2, arrows=True, arrowsize=8, arrowstyle='->',
                     node_color=node_colors_louvain)

plt.title("Blogosphere grouped by Louvain", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

## community detection

In [ ]:
# Plot network with Leiden communities
plt.figure(figsize=(15, 12))

# Use the same layout as before for comparison
pos = nx.draw_kamada_kawai(subgraph_undirected)

# Prepare node colors based on Leiden communities
n_leiden_communities = len(set(leiden_partition.membership))
leiden_colors = plt.cm.Set1(np.linspace(0, 1, n_leiden_communities))
node_colors_leiden = []
for node in subgraph_undirected.nodes():
    community_id = group_leiden.get(node, 0)
    node_colors_leiden.append(leiden_colors[community_id % len(leiden_colors)])

# Use same node sizes as before
# node_sizes_leiden = [in_degrees_full.get(node, 1) * 15 + 30 for node in subgraph_undirected.nodes()]

# Draw the network
nx.draw_networkx_edges(subgraph_undirected, pos, edge_color='gray', alpha=0.3, width=0.2,
                      arrows=True, arrowsize=8, arrowstyle='->')
nx.draw_networkx_nodes(subgraph_undirected, pos, node_color=node_colors_leiden, 
                       alpha=0.8, edgecolors='black', linewidths=0.5)

plt.title("Blogosphere grouped by Leiden", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

# Bluesky: What is my Bluesky network?

Before we start working with Bluesky, you should authenticate your
session:

In [ ]:
client = Client()
client.login('jbgruber.bsky.social', '4srg-qdme-rgsr-cmal')

When you are new to Bluesky you should start by searching a few names
that you know. There are also starter packs like
<https://bsky.app/starter-pack/sof14g1l.bsky.social/3lbc4bqetfp22> which
you can follow. But what then? Given the idea of homophily, you might
want to check who the people you follow, follow themselves. So let’s get
started with that (replace my handle with yours below if you like):

In [ ]:
# Get my follows (replace with your handle)
def get_follows_data(handle, limit=None):
    """Get follows for a given handle"""
    try:
        response = client.get_follows(handle, limit=limit)
        follows_data = []
        for follow in response.follows:
            follows_data.append({
                'actor_handle': follow.handle,
                'actor_did': follow.did,
                'actor_display_name': follow.display_name or ''
            })
        return pd.DataFrame(follows_data)
    except Exception as e:
        print(f"Error getting follows for {handle}: {e}")
        return pd.DataFrame()

my_follows = get_follows_data("jbgruber.bsky.social")
print(f"Shape of my_follows: {my_follows.shape}")
print(my_follows.head())

Next, we want to see who they follow:

In [ ]:
def get_their_follows(handles, limit=100):
    """Get follows for multiple handles"""
    their_follows = []
    
    for handle in tqdm(handles, desc="Getting follows"):
        follows_df = get_follows_data(handle, limit=limit)
        if not follows_df.empty:
            follows_df['from'] = handle
            follows_df['to'] = follows_df['actor_handle']
            their_follows.append(follows_df[['from', 'to']])
    
    return pd.concat(their_follows, ignore_index=True) if their_follows else pd.DataFrame()

# Save/load data to avoid repeated API calls
their_follows_file = "data/their_follows_py.pkl"
os.makedirs("data", exist_ok=True)

if not os.path.exists(their_follows_file):
    # Take last 50 accounts I followed
    recent_follows = my_follows['actor_handle'].tail(50).tolist()
    their_follows = get_their_follows(recent_follows)
    
    with open(their_follows_file, 'wb') as f:
        pickle.dump(their_follows, f)
else:
    with open(their_follows_file, 'rb') as f:
        their_follows = pickle.load(f)

print(f"Shape of their_follows: {their_follows.shape}")
print(their_follows.head())

As a first step, we can just check the raw number of who shows up most
often here:

In [ ]:
# Check raw counts of who shows up most often
follow_counts = their_follows['to'].value_counts()
follow_counts = follow_counts[follow_counts.index != 'handle.invalid']
top_15 = follow_counts.head(15)

plt.figure(figsize=(10, 8))
plt.barh(range(len(top_15)), top_15.values)
plt.yticks(range(len(top_15)), top_15.index)
plt.xlabel('Count')
plt.title('Most Followed Accounts')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

But we can also use network analysis to find influential accounts:

In [ ]:
# Create network graph
follows_graph = nx.from_pandas_edgelist(their_follows, 'from', 'to', create_using=nx.DiGraph())

# Calculate centrality measures
degree_centrality = nx.degree_centrality(follows_graph)
closeness_centrality = nx.closeness_centrality(follows_graph)
betweenness_centrality = nx.betweenness_centrality(follows_graph)
eigenvector_centrality = nx.eigenvector_centrality(follows_graph, max_iter=1000)

# Create centrality dataframe
centrality_df = pd.DataFrame({
    'name': list(follows_graph.nodes()),
    'degree': [degree_centrality[node] for node in follows_graph.nodes()],
    'closeness': [closeness_centrality[node] for node in follows_graph.nodes()],
    'betweenness': [betweenness_centrality[node] for node in follows_graph.nodes()],
    'eigenvector': [eigenvector_centrality[node] for node in follows_graph.nodes()]
})

Let’s look at **degree** centrality (simply counts the number of
neighbors a node has):

In [ ]:
print("Top 10 by degree centrality:")
print(centrality_df.nlargest(10, 'degree')[['name', 'degree']])

**closeness** (computes the shortest path distances among nodes. The
most central node has the minimum distance to all other nodes)

In [ ]:
print("Top 10 by closeness centrality:")
print(centrality_df.nlargest(10, 'closeness')[['name', 'closeness']])

**betweeness** (number of shortest paths that pass through a node,
divided by the total number of shortest paths)

In [ ]:
print("Top 10 by betweenness centrality:")
central_accounts = centrality_df.nlargest(10, 'betweenness')
print(central_accounts[['name', 'betweenness']])

and **eigenvector centrality** (extends the idea of degree by assuming
that a node is central if it is connected to other central nodes)

In [ ]:
print("Top 10 by eigenvector centrality:")
print(centrality_df.nlargest(10, 'eigenvector')[['name', 'eigenvector']])

We can also visialise this network and highlight some of the accounts in
it:

In [ ]:
# Create network visualization highlighting central accounts
plt.figure(figsize=(15, 12))

# Get top 500 nodes by eigenvector centrality for visualization
top_nodes = centrality_df.nlargest(500, 'eigenvector')['name'].tolist()
subgraph = follows_graph.subgraph(top_nodes)

# Get central accounts for highlighting
central_account_names = central_accounts['name'].tolist()

# Create layout
pos = nx.spring_layout(subgraph, k=1, iterations=50)

# Draw edges
nx.draw_networkx_edges(subgraph, pos, alpha=0.3, width=0.5, edge_color='gray')

# Draw nodes
node_colors = ['firebrick' if node in central_account_names else 'lightblue' 
               for node in subgraph.nodes()]
node_sizes = [centrality_df[centrality_df['name'] == node]['betweenness'].iloc[0] * 3000 + 50 
              for node in subgraph.nodes()]

nx.draw_networkx_nodes(subgraph, pos, node_color=node_colors, node_size=node_sizes, alpha=0.8)

# Add labels for central accounts
labels = {node: node if node in central_account_names else '' for node in subgraph.nodes()}
nx.draw_networkx_labels(subgraph, pos, labels, font_size=8)

plt.title('Network of Follows (Central Accounts Highlighted)', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

For a different approach to find people to follow, you can check out
<https://www.johannesbgruber.eu/post/2024-11-24-bluesky-rising/>.

# Bluesky: Checking out the #rstats network

First let’s get some data. The code below searches for posts that
mention the hashtag #rstats, which is widely use for all things R:

In [ ]:
def search_posts(query, since_date=None, limit=None):
    """Search for posts with specific query using pagination"""
    posts_data = []
    total_limit = limit or 100
    req_limit = min(total_limit, 100)  # API max is 100 per request
    cursor = None
    
    try:
        while len(posts_data) < total_limit:
            # Adjust request limit for final batch
            current_limit = min(req_limit, total_limit - len(posts_data))
            
            # Prepare search parameters
            search_params = {
                'q': query,
                'limit': current_limit
            }
            
            # Add cursor for pagination if available
            if cursor:
                search_params['cursor'] = cursor
            
            # Add since date if provided
            if since_date:
                search_params['since'] = since_date
            
            # Make API call
            response = client.app.bsky.feed.search_posts(search_params)
            
            # Extract posts from response
            if hasattr(response, 'posts') and response.posts:
                for post in response.posts:
                    # Extract hashtags from text
                    hashtags = re.findall(r'#\w+', post.record.text.lower())
                    
                    posts_data.append({
                        'cid': post.cid,
                        'uri': post.uri,
                        'author_handle': post.author.handle,
                        'text': post.record.text,
                        'tags': hashtags,
                        'reply_count': getattr(post, 'reply_count', 0) or 0,
                        'repost_count': getattr(post, 'repost_count', 0) or 0,
                        'like_count': getattr(post, 'like_count', 0) or 0
                    })
            else:
                # No more posts available
                break
            
            # Get cursor for next batch
            if hasattr(response, 'cursor') and response.cursor:
                cursor = response.cursor
            else:
                # No more pages available
                break
            
            # Optional: show progress
            if len(posts_data) % 100 == 0:
                print(f"Retrieved {len(posts_data)} posts so far...")
        
        print(f"Total posts retrieved: {len(posts_data)}")
        return pd.DataFrame(posts_data)
        
    except Exception as e:
        print(f"Error searching posts: {e}")
        print(f"Retrieved {len(posts_data)} posts before error")
        return pd.DataFrame(posts_data) if posts_data else pd.DataFrame()

# Save/load data
rstats_content_file = "data/rstats_content_py.pkl"

if not os.path.exists(rstats_content_file):
    rstats_content = search_posts("#rstats", since_date="2025-04-01T00:00:00.000Z", limit=math.inf)
    with open(rstats_content_file, 'wb') as f:
        pickle.dump(rstats_content, f)
else:
    with open(rstats_content_file, 'rb') as f:
        rstats_content = pickle.load(f)

print(f"Shape of rstats_content: {rstats_content.shape}")
print(rstats_content.head())

## Semantic Network/Co-hashtag Network

The first network we can build from this data is a co-occurence network.
We check, which hashtags are used together in these posts. In other
words: the hashtags are nodes, and the edges are whether the hashtags
were used together. As a first step, let’s get some info on the most
popular hashtags:

In [ ]:
# Extract and count hashtags
rstats_tags = []
for idx, row in rstats_content.iterrows():
    for tag in row['tags']:
        rstats_tags.append({
            'cid': row['cid'],
            'hashtag': tag.lower()
        })

rstats_tags_df = pd.DataFrame(rstats_tags)
rstats_tags_count = rstats_tags_df['hashtag'].value_counts().reset_index()
rstats_tags_count.columns = ['hashtag', 'n']

# Plot top hashtags
plt.figure(figsize=(10, 8))
top_hashtags = rstats_tags_count.head(10)
plt.barh(range(len(top_hashtags)), top_hashtags['n'])
plt.yticks(range(len(top_hashtags)), top_hashtags['hashtag'])
plt.xlabel('Count')
plt.title('Top Hashtags in #rstats Posts')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

Now, to find co-occurrence of hashtags. You could also do this with
words, but it often comes out less meaningful.

In [ ]:
cooccurrence_pairs = []
for cid in rstats_tags_df['cid'].unique():
    post_tags = rstats_tags_df[rstats_tags_df['cid'] == cid]['hashtag'].tolist()
    if len(post_tags) > 1:
        for pair in combinations(post_tags, 2):
            cooccurrence_pairs.append({'from': pair[0], 'to': pair[1]})
            cooccurrence_pairs.append({'from': pair[1], 'to': pair[0]})  # Undirected

rstats_tags_cooc = pd.DataFrame(cooccurrence_pairs)
print(f"Co-occurrence pairs: {len(rstats_tags_cooc)}")
print(rstats_tags_cooc.head())

Now you should notice this looks like a network already. Let’s make it
one:

In [ ]:
# Create hashtag network
hashtag_network = nx.from_pandas_edgelist(rstats_tags_cooc, 'from', 'to')

# Add node attributes (counts)
hashtag_counts_dict = dict(zip(rstats_tags_count['hashtag'], rstats_tags_count['n']))
nx.set_node_attributes(hashtag_network, hashtag_counts_dict, 'count')

print(f"Hashtag network: {hashtag_network.number_of_nodes()} nodes, {hashtag_network.number_of_edges()} edges")

We can try a few things to turn this into a nice plot. Let’s get a
baseline first:

In [ ]:
# Get top 50 hashtags for visualization
top_50_hashtags = rstats_tags_count.head(50)['hashtag'].tolist()
hashtag_subgraph = hashtag_network.subgraph(top_50_hashtags)

plt.figure(figsize=(15, 12))
pos = nx.spring_layout(hashtag_subgraph, k=2, iterations=50)

# Draw network
nx.draw_networkx_edges(hashtag_subgraph, pos, alpha=0.3, width=0.5)
nx.draw_networkx_labels(hashtag_subgraph, pos, font_size=8)

plt.title('Co-occurrence Network of #rstats Hashtags (Top 50)', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

We can use color to show the centrality of nodes. You can play around
with which measure produces the most interesting visual.

In [ ]:
# Calculate centrality measures for hashtag network
hashtag_centrality = {
    'degree': nx.degree_centrality(hashtag_subgraph),
    'closeness': nx.closeness_centrality(hashtag_subgraph),
    'betweenness': nx.betweenness_centrality(hashtag_subgraph),
    'eigenvector': nx.eigenvector_centrality(hashtag_subgraph, max_iter=1000)
}

# Create visualization with eigenvector centrality coloring
plt.figure(figsize=(15, 12))
pos = nx.spring_layout(hashtag_subgraph, k=2, iterations=50)

# Color nodes by eigenvector centrality
eigenvector_values = [hashtag_centrality['eigenvector'][node] for node in hashtag_subgraph.nodes()]
nodes = nx.draw_networkx_nodes(hashtag_subgraph, pos, 
                              node_color=eigenvector_values, 
                              cmap='coolwarm', 
                              node_size=300)

nx.draw_networkx_edges(hashtag_subgraph, pos, alpha=0.3, width=0.5)
nx.draw_networkx_labels(hashtag_subgraph, pos, font_size=8)

plt.colorbar(nodes, label='Eigenvector Centrality')
plt.title('Hashtag Network Colored by Eigenvector Centrality', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

We can also see if we can find some communities in this data. As above,
we are using the Louvain and Leiden algorithms:

In [ ]:
# import networkx.algorithms.community as nx_comm

louvain_communities = nx.community.louvain_communities(hashtag_subgraph, seed=42)
# leiden_communities = nx.community.leiden_communities(hashtag_subgraph, seed=42) 

# Create community assignments
louvain_dict = {}
for i, community in enumerate(louvain_communities):
    for node in community:
        louvain_dict[node] = i

leiden_dict = {}
# for i, community in enumerate(leiden_communities):
#     for node in community:
#         leiden_dict[node] = i

print(f"Louvain found {len(louvain_communities)} communities")
# print(f"Leiden found {len(leiden_communities)} communities")

Here is the network grouped by Louvain:

In [ ]:
plt.figure(figsize=(15, 12))
pos = nx.spring_layout(hashtag_subgraph, k=2, iterations=50)

# Color nodes by Louvain communities
community_colors = plt.cm.Set3(np.linspace(0, 1, len(louvain_communities)))
node_colors = [community_colors[louvain_dict.get(node, 0)] for node in hashtag_subgraph.nodes()]

nx.draw_networkx_nodes(hashtag_subgraph, pos, node_color=node_colors, node_size=300)
nx.draw_networkx_edges(hashtag_subgraph, pos, alpha=0.3, width=0.5)
nx.draw_networkx_labels(hashtag_subgraph, pos, font_size=8)

plt.title('Hashtag Network - Louvain Communities', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

And here it is grouped by the Leiden algorithm:

``` python
plt.figure(figsize=(15, 12))
pos = nx.spring_layout(hashtag_subgraph, k=2, iterations=50)

# Color nodes by Leiden communities
community_colors = plt.cm.Set1(np.linspace(0, 1, len(leiden_communities)))
node_colors = [community_colors[leiden_dict.get(node, 0)] for node in hashtag_subgraph.nodes()]

nx.draw_networkx_nodes(hashtag_subgraph, pos, node_color=node_colors, node_size=300)
nx.draw_networkx_edges(hashtag_subgraph, pos, alpha=0.3, width=0.5)
nx.draw_networkx_labels(hashtag_subgraph, pos, font_size=8)

plt.title('Hashtag Network - Leiden Communities', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()
```

## Follower Network

Within the #rstats content, we can also check the community that posts
this content. We first get the user handles from the data:

In [ ]:
# Get unique users from rstats content
rstats_users = rstats_content['author_handle'].unique()
print(f"Unique #rstats users: {len(rstats_users)}")

Now we query who follows these users who post about #rstats:

In [ ]:
def get_followers_data(handle, limit=100):
    """Get followers for a given handle"""
    try:
        response = client.get_followers(handle, limit=limit)
        followers = [follow.handle for follow in response.followers]
        return followers
    except Exception as e:
        print(f"Error getting followers for {handle}: {e}")
        return []

# Save/load follower data
follower_data_file = "data/follower_data_py.pkl"

if not os.path.exists(follower_data_file):
    follower_data = []
    for user in tqdm(rstats_users[:50], desc="Getting followers"):  # Limit for demo
        followers = get_followers_data(user)
        follower_data.append({
            'author_handle': user,
            'followed_by': followers
        })
    
    follower_df = pd.DataFrame(follower_data)
    with open(follower_data_file, 'wb') as f:
        pickle.dump(follower_df, f)
else:
    with open(follower_data_file, 'rb') as f:
        follower_df = pickle.load(f)

print(f"Follower data shape: {follower_df.shape}")

Who follows the most accounts contributing to #rstats?

In [ ]:
# Who follows the most #rstats accounts?
all_followers = []
for idx, row in follower_df.iterrows():
    for follower in row['followed_by']:
        all_followers.append(follower)

follower_counts = pd.Series(all_followers).value_counts().head(10)

plt.figure(figsize=(10, 8))
plt.barh(range(len(follower_counts)), follower_counts.values)
plt.yticks(range(len(follower_counts)), follower_counts.index)
plt.xlabel('Number of #rstats accounts followed')
plt.title('Users who follow the most #rstats accounts')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

We can turn this data into a directed network where the nodes are users
and the edges are whether a user follows another:

In [ ]:
# Create follower network edges
follower_edges = []
for idx, row in follower_df.iterrows():
    for follower in row['followed_by']:
        follower_edges.append({
            'from': follower,
            'to': row['author_handle']
        })

follower_network_df = pd.DataFrame(follower_edges)
follower_network = nx.from_pandas_edgelist(follower_network_df, 'from', 'to', create_using=nx.DiGraph())

print(f"Follower network: {follower_network.number_of_nodes()} nodes, {follower_network.number_of_edges()} edges")

In [ ]:
# not a good idea to actually run this, since networkX is so slow
plt.figure(figsize=(15, 12))
nx.draw_kamada_kawai(follower_network)
plt.show()

This is a mess, since there are so many nodes in the network now. We can
again look for the most central nodes (aka users) in this network:

In [ ]:
# Calculate centrality measures for follower network
follower_centrality = {
    'degree': nx.degree_centrality(follower_network),
    'closeness': nx.closeness_centrality(follower_network),
    'betweenness': nx.betweenness_centrality(follower_network),
    'eigenvector': nx.eigenvector_centrality(follower_network, max_iter=1000)
}

# Create centrality dataframe
follower_centrality_df = pd.DataFrame({
    'name': list(follower_network.nodes()),
    'degree': [follower_centrality['degree'][node] for node in follower_network.nodes()],
    'closeness': [follower_centrality['closeness'][node] for node in follower_network.nodes()],
    'betweenness': [follower_centrality['betweenness'][node] for node in follower_network.nodes()],
    'eigenvector': [follower_centrality['eigenvector'][node] for node in follower_network.nodes()]
})

print("Most central users in follower network:")
print(follower_centrality_df.nlargest(10, 'degree')[['name', 'degree']])

Now let’s try only the most central accounts:

In [ ]:
# Visualize follower network (top 100 nodes by eigenvector centrality)
top_100_nodes = follower_centrality_df.nlargest(100, 'eigenvector')['name'].tolist()
most_central_nodes = follower_centrality_df.nlargest(5, 'eigenvector')['name'].tolist()

follower_subgraph = follower_network.subgraph(top_100_nodes)

plt.figure(figsize=(15, 12))
pos = nx.spring_layout(follower_subgraph, k=1, iterations=50)

# Draw edges
nx.draw_networkx_edges(follower_subgraph, pos, alpha=0.3, width=0.5, edge_color='gray')

# Draw nodes
nx.draw_networkx_nodes(follower_subgraph, pos, node_color='firebrick', node_size=50, alpha=0.8)

# Add labels for most central nodes
labels = {node: node if node in most_central_nodes else '' for node in follower_subgraph.nodes()}
nx.draw_networkx_labels(follower_subgraph, pos, labels, font_size=8)

plt.title('Follower Network (#rstats users)', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

## Mention Network

We can also contruct a network with users as nodes and mentions as the
edges. For this, let’s extract mentions first:

In [ ]:
# Extract mentions from rstats content
mention_pattern = r'@(\w+)'
mention_edges = []

for idx, row in rstats_content.iterrows():
    if '@' in row['text']:
        mentions = re.findall(mention_pattern, row['text'])
        for mention in mentions:
            mention_edges.append({
                'from': row['author_handle'],
                'to': f"@{mention}"
            })

mentions_df = pd.DataFrame(mention_edges)
print(f"Mention edges: {len(mentions_df)}")
print(mentions_df.head())

Let’s see who is mentioned most:

In [ ]:
# Who is mentioned most?
mention_counts = mentions_df['to'].value_counts().head(10)

plt.figure(figsize=(10, 8))
plt.barh(range(len(mention_counts)), mention_counts.values)
plt.yticks(range(len(mention_counts)), mention_counts.index)
plt.xlabel('Mention Count')
plt.title('Most Mentioned Users in #rstats Posts')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

The network comes natural at this point:

In [ ]:
# Create and visualize mention network
mention_network = nx.from_pandas_edgelist(mentions_df, 'from', 'to', create_using=nx.DiGraph())

plt.figure(figsize=(15, 12))
pos = nx.spring_layout(mention_network, k=1, iterations=50)

nx.draw_networkx_edges(mention_network, pos, alpha=0.3, width=0.5, edge_color='gray')
nx.draw_networkx_nodes(mention_network, pos, node_color='firebrick', node_size=30, alpha=0.8)

plt.title('Mention Network in #rstats Posts', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

## Share/repost Network

Another way to contruct a network from this data is by looking for
reposts. Here the nodes would be posts

In [ ]:
def get_reposts_data(uri, limit=100):
    """Get reposts for a specific post URI"""
    try:
        response = client.app.bsky.feed.get_reposted_by({"uri": uri, "limit": limit})
        # reposts = response.reposted_by
        reposts = [repost.handle for repost in response.reposted_by]
        return reposts
    except Exception as e:
        print(f"Error getting reposts for {uri}: {e}")
        return []

# Get repost data for posts with replies
repost_data_file = "data/repost_data_py.pkl"

if not os.path.exists(repost_data_file):
    repost_data = []
    posts_with_replies = rstats_content[rstats_content['reply_count'] > 0]
    
    for idx, row in tqdm(posts_with_replies.head(20).iterrows(), desc="Getting reposts"):
        reposts = get_reposts_data(row['uri'])
        if reposts:
            repost_data.append({
                'uri': row['uri'],
                'author_handle': row['author_handle'],
                'reposted_by': reposts
            })
    
    repost_df = pd.DataFrame(repost_data)
    with open(repost_data_file, 'wb') as f:
        pickle.dump(repost_df, f)
else:
    with open(repost_data_file, 'rb') as f:
        repost_df = pickle.load(f)

print(f"Repost data shape: {repost_df.shape}")

In [ ]:
# Create repost network edges
repost_edges = []
for idx, row in repost_df.iterrows():
    for reposter in row['reposted_by']:
        repost_edges.append({
            'from': row['author_handle'],
            'to': reposter
        })

repost_network_df = pd.DataFrame(repost_edges)
repost_network = nx.from_pandas_edgelist(repost_network_df, 'from', 'to', create_using=nx.DiGraph())

print(f"Repost network: {repost_network.number_of_nodes()} nodes, {repost_network.number_of_edges()} edges")

In [ ]:
plt.figure(figsize=(12, 10))
pos = nx.spring_layout(repost_network, k=1, iterations=50)

nx.draw_networkx_edges(repost_network, pos, alpha=0.3, width=0.5, edge_color='gray')
nx.draw_networkx_nodes(repost_network, pos, node_color='firebrick', node_size=50, alpha=0.8)

plt.title('Repost Network (#rstats)', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

Let’s look at centrality again to see

In [ ]:
# Calculate centrality for repost network
repost_centrality = {
    'degree': nx.degree_centrality(repost_network),
    'closeness': nx.closeness_centrality(repost_network),
    'betweenness': nx.betweenness_centrality(repost_network),
    'eigenvector': nx.eigenvector_centrality(repost_network, max_iter=1000)
}

repost_centrality_df = pd.DataFrame({
    'name': list(repost_network.nodes()),
    'degree': [repost_centrality['degree'][node] for node in repost_network.nodes()],
    'closeness': [repost_centrality['closeness'][node] for node in repost_network.nodes()],
    'betweenness': [repost_centrality['betweenness'][node] for node in repost_network.nodes()],
    'eigenvector': [repost_centrality['eigenvector'][node] for node in repost_network.nodes()]
})

print("Most central users in repost network:")
most_central_repost = repost_centrality_df.nlargest(5, 'degree')
print(most_central_repost[['name', 'degree']])

Let’s once again see which are the most central accounts:

In [ ]:
# Visualize with top nodes highlighted
top_100_repost = repost_centrality_df.nlargest(100, 'degree')['name'].tolist()
most_central_names = most_central_repost['name'].tolist()

if top_100_repost:
    repost_subgraph = repost_network.subgraph(top_100_repost)
    
    plt.figure(figsize=(15, 12))
    pos = nx.spring_layout(repost_subgraph, k=1, iterations=50)
    
    # Draw edges
    nx.draw_networkx_edges(repost_subgraph, pos, alpha=0.3, width=0.5, edge_color='gray')
    
    # Draw nodes
    nx.draw_networkx_nodes(repost_subgraph, pos, node_color='firebrick', node_size=50, alpha=0.8)
    
    # Add labels for most central nodes
    labels = {node: node if node in most_central_names else '' for node in repost_subgraph.nodes()}
    nx.draw_networkx_labels(repost_subgraph, pos, labels, font_size=8)
    
    plt.title('Repost Network with Central Users Highlighted', fontsize=16)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

Let’s see if we can find any sensible communities in this network:

In [ ]:
# Community detection for repost network
if repost_network.number_of_nodes() > 0:
    # Convert to undirected for community detection
    repost_undirected = repost_network.to_undirected()
    
    if repost_undirected.number_of_nodes() > 1:
        louvain_repost = nx_comm.louvain_communities(repost_undirected, seed=42)
        
        # Create community visualization
        plt.figure(figsize=(12, 10))
        pos = nx.spring_layout(repost_undirected, k=1, iterations=50)
        
        # Color nodes by community
        community_colors = plt.cm.Set3(np.linspace(0, 1, len(louvain_repost)))
        louvain_repost_dict = {}
        for i, community in enumerate(louvain_repost):
            for node in community:
                louvain_repost_dict[node] = i
        
        node_colors = [community_colors[louvain_repost_dict.get(node, 0)] 
                      for node in repost_undirected.nodes()]
        
        nx.draw_networkx_edges(repost_undirected, pos, alpha=0.3, width=0.5, edge_color='gray')
        nx.draw_networkx_nodes(repost_undirected, pos, node_color=node_colors, node_size=50, alpha=0.8)
        
        # Add labels for most central nodes
        labels = {node: node if node in most_central_names else '' for node in repost_undirected.nodes()}
        nx.draw_networkx_labels(repost_undirected, pos, labels, font_size=8)
        
        plt.title('Repost Network with Louvain Communities', fontsize=16)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        
        print(f"Found {len(louvain_repost)} communities in repost network")
    else:
        print("Not enough nodes for community detection in repost network")
else:
    print("No repost network data available")